In [118]:
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
from dotenv import load_dotenv
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import os
import time
import json
import joblib


In [119]:
HELIUS_API_KEY = os.getenv('HELIUM_API_KEY')


In [120]:
VALIDATORS_API_KEY = os.getenv('VALIDATORS_API_KEY')

In [121]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)   
pd.set_option('display.width', 200)    
pd.set_option('display.float_format', '{:.8f}'.format)

In [122]:
# transactions_api  = TransactionsAPI(HELIUS_API_KEY)

# parsed_transaction_history = transactions_api.get_parsed_transaction_history(address="HtXa1PH33GGvH3giqMqatndHcKnzeSkwMaW46DTzDfLd")

# print(parsed_transaction_history)

In [123]:
import requests

# Replace this with your Helius API key
#API_KEY = "YOUR_HELIUS_API_KEY"
BASE_URL = f"https://mainnet.helius-rpc.com/?api-key={HELIUS_API_KEY}"

HEADERS = {"Content-Type": "application/json"}

def get_vote_accounts():
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getVoteAccounts"
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        current_validators = data["result"]["current"]
        delinquent_validators = data["result"]["delinquent"]
        return current_validators, delinquent_validators
    else:
        print("Error fetching vote accounts:", response.text)
        return [], []

def get_stake_accounts_by_pubkey(pubkey):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getProgramAccounts",
        "params": [
            "Stake11111111111111111111111111111111111111",  # Stake Program
            {
                "encoding": "jsonParsed",
                "filters": [
                    {
                        "memcmp": {
                            "offset": 12,
                            "bytes": pubkey
                        }
                    }
                ]
            }
        ]
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        return response.json()["result"]
    else:
        print("Error fetching stake accounts:", response.text)
        return []



In [124]:
# # --- Example Usage ---

# # Fetch validators
# current, delinquent = get_vote_accounts()
# print(f"Total active validators: {len(current)}")
# print(f"Total delinquent validators: {len(delinquent)}")

# # Optionally fetch stake accounts by a delegator pubkey
# delegator_pubkey = "HtXa1PH33GGvH3giqMqatndHcKnzeSkwMaW46DTzDfLd"  # Replace this
# stakes = get_stake_accounts_by_pubkey(delegator_pubkey)
# print(f"Stake accounts for {delegator_pubkey}:")
# for s in stakes:
#     print(s["pubkey"], s["account"]["data"]["parsed"]["info"]["stake"])


# Fetching Validator + Stake Data via Helius API

### Vote Data

In [125]:
# JSON-RPC Payload to fetch validator vote accounts
payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "getVoteAccounts"
}

response = requests.post(BASE_URL, json=payload)
data = response.json()

# Combine current + delinquent validators
validators = data["result"]["current"] + data["result"]["delinquent"]

# Convert to DataFrame
df = pd.DataFrame(validators)
df.head(n=1)

,activatedStake,commission,epochCredits,epochVoteAccount,lastVote,nodePubkey,rootSlot,votePubkey
0,763095337836965,0,"[[782, 359374327, 352476184], [783, 366264519, 359374327], [784, 373162756, 366264519], [785, 380063290, 373162756], [786, 381504501, 380063290]]",True,339642420,HM1KjNaXa4w8K4gCXbieoMh5gUTNeUhg9fvdXMKeBW3L,339642389,HMV14UAuULSwqmZhsKHzaVkYAd94iWpEeURgbUegfQLc


In [126]:
# Add additional fields (can be joined with price data or slashing reports)
df["timestamp"] = pd.Timestamp.utcnow()
df["epoch"] = None  # You can get this via 'getEpochInfo' method

# Preview important fields
# print(df[[
#     "nodePubkey", "commission", "activatedStake", "lastVote", "rootSlot", 
#     "epochCredits", "epochVoteAccount"
# ]])

df = df[[
    "timestamp","nodePubkey", "commission", "activatedStake", "lastVote", "rootSlot", 
    "epochCredits", "epochVoteAccount","votePubkey"
]]


In [127]:
print(df.columns.tolist())


['timestamp', 'nodePubkey', 'commission', 'activatedStake', 'lastVote', 'rootSlot', 'epochCredits', 'epochVoteAccount', 'votePubkey']


In [128]:
expanded_rows = []

for idx, row in df.iterrows():
    node_pubkey = row['nodePubkey']
    votePubkey = row['votePubkey']
    for epoch_info in row['epochCredits']:
        epoch, current_credits, previous_credits = epoch_info
        credits_earned = current_credits - previous_credits
        expanded_rows.append({
            'timestamp': row['timestamp'],
            'nodePubkey': node_pubkey,
            'commission': row['commission'],
            'activatedStake': row['activatedStake'],
            'lastVote': row['lastVote'],
            'rootSlot': row['rootSlot'],
            'epochVoteAccount': row['epochVoteAccount'],
            'epoch': epoch,
            'credits': current_credits,
            'previous_credits': previous_credits,
            'credits_earned': credits_earned,
            'votePubkey' : votePubkey
        })

df_expanded = pd.DataFrame(expanded_rows)
df_expanded.sort_values(by=['nodePubkey', 'epoch'], inplace=True)
print(df_expanded.head())


# Preferred: use .joblib extension
joblib.dump(df_expanded, 'df_expanded.joblib')


                            timestamp                                    nodePubkey  commission  activatedStake   lastVote   rootSlot  epochVoteAccount  epoch    credits  previous_credits  \
5852 2025-05-13 01:26:26.561946+00:00  138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT           0  28931907684368  339642420  339642389              True    782  663485837         656598333   
5853 2025-05-13 01:26:26.561946+00:00  138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT           0  28931907684368  339642420  339642389              True    783  670375331         663485837   
5854 2025-05-13 01:26:26.561946+00:00  138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT           0  28931907684368  339642420  339642389              True    784  677274232         670375331   
5855 2025-05-13 01:26:26.561946+00:00  138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT           0  28931907684368  339642420  339642389              True    785  684174907         677274232   
5856 2025-05-13 01:26:26.561946+00:00  138KHw

['df_expanded.joblib']

In [129]:
df_expanded["epoch"].unique() #.sum()

array([782, 783, 784, 785, 786, 778, 779, 780, 781, 777, 773, 774, 775,
       776, 744, 745, 746, 769, 770, 772, 766])

### Annual Inflation Rate
Purpose: Use this to contextualize staking APY and understand network reward distribution.





In [130]:
def get_inflation_rate():
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getInflationRate"
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        #return data["result"]
        result = data["result"]
        result["timestamp"] = pd.Timestamp.utcnow()  # Add timestamp
        return result 
    else:
        print("Error fetching inflation rate:", response.text)
        return {}

# Fetch and convert to DataFrame
inflation_data = get_inflation_rate()
df_inflation = pd.DataFrame([inflation_data])  # Single row
print(df_inflation) #.head())

joblib.dump(df_inflation, 'df_inflation.joblib')

   epoch  foundation      total  validator                        timestamp
0    786  0.00000000 0.04538333 0.04538333 2025-05-13 01:26:27.260633+00:00


['df_inflation.joblib']

### Staking Rewards - Inflation Rate
This is critical for calculating APY


- amount: Rewards in lamports (e.g., 1,863,991,600 lamports ≈ 1.863 SOL).

- effectiveSlot: Slot when rewards were applied.

- epoch: Epoch number.

- postBalance: Account balance after rewards (in lamports).

- commission: Validator’s commission rate




In [131]:
def get_inflation_reward(vote_accounts, epoch=None):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getInflationReward",
        "params": [vote_accounts, {"commitment": "finalized"}]
    }
    if epoch is not None:
        payload["params"].append({"epoch": epoch})
        
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    
    if response.status_code == 200:
        data = response.json()
        result = data["result"]
        
        # Add timestamp to each reward entry
        timestamp = pd.Timestamp.utcnow()
        for entry in result:
            if entry is not None:  # Some entries may be None
                entry["timestamp"] = timestamp
        
        return result
    else:
        print("Error fetching inflation reward:", response.text)
        return []


# Get validator-specific inflation rewards
validator_addresses = df_expanded["votePubkey"].tolist()
rewards = get_inflation_reward(validator_addresses[:100])  # Max 50 addresses per call

inflation_reward = pd.DataFrame(rewards)
#inflation_reward = inflation_reward[inflation_reward["amount"] > 0]
print(inflation_reward)

joblib.dump(inflation_reward, 'inflation_reward.joblib')

       amount  commission  effectiveSlot  epoch  postBalance                        timestamp
0           0           0      339552000    785     31194720 2025-05-13 01:26:29.057580+00:00
1           0           0      339552000    785     31194720 2025-05-13 01:26:29.057580+00:00
2           0           0      339552000    785     31194720 2025-05-13 01:26:29.057580+00:00
3           0           0      339552000    785     31194720 2025-05-13 01:26:29.057580+00:00
4           0           0      339552000    785     31194720 2025-05-13 01:26:29.057580+00:00
..        ...         ...            ...    ...          ...                              ...
95  362205784           5      339552000    785    956067505 2025-05-13 01:26:29.057580+00:00
96  362205784           5      339552000    785    956067505 2025-05-13 01:26:29.057580+00:00
97  362205784           5      339552000    785    956067505 2025-05-13 01:26:29.057580+00:00
98  362205784           5      339552000    785    956067505

['inflation_reward.joblib']

### Total Network Credits

Use total network credits to normalize your validator’s credits_earned and assess relative performance. Also provides epoch context.



In [132]:
def get_epoch_info():
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getEpochInfo"
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        result = data["result"]
        result["timestamp"] = pd.Timestamp.utcnow()  # Add timestamp
        return result 
    else:
        print("Error fetching epoch info:", response.text)
        return {}

# Fetch and convert to DataFrame
epoch_info = get_epoch_info()
df_epoch = pd.DataFrame([epoch_info])
print(df_epoch) #.head())


joblib.dump(df_epoch, 'df_epoch.joblib')

   absoluteSlot  blockHeight  epoch  slotIndex  slotsInEpoch  transactionCount                        timestamp
0     339642431    317864333    786      90431        432000      405331757367 2025-05-13 01:26:29.736464+00:00


['df_epoch.joblib']

## Circulating SOL Supply



In [133]:
def get_supply():
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getSupply",
        "params": [{"commitment": "finalized"}]
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        return data["result"]["value"]
    else:
        print("Error fetching supply:", response.text)
        return {}

# Fetch and convert to DataFrame
supply_data = get_supply()
df_supply = pd.DataFrame([supply_data])
print(df_supply.head(n=1))



          circulating     nonCirculating  \
0  519444081879735205  81207422190013737   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [134]:
import pandas as pd

# If df_supply is already created:
df_supply['circulating'] = df_supply['circulating'].astype(float)
df_supply['nonCirculating'] = df_supply['nonCirculating'].astype(float)
df_supply['total'] = df_supply['total'].astype(float)
# Optionally convert to billions for readability:
df_supply['circulating_sol'] = df_supply['circulating'] / 1e9
df_supply['nonCirculating_sol'] = df_supply['nonCirculating'] / 1e9
df_supply['total_sol'] = df_supply['total'] / 1e9
# If you want a DataFrame where each nonCirculatingAccount is its own row:
df_accounts = pd.DataFrame(df_supply['nonCirculatingAccounts'][0], columns=['nonCirculatingAccount'])

# Resulting outputs:
df_supply = df_supply[['circulating_sol', 'nonCirculating_sol', 'total_sol']]
print(df_supply)
#print(df_supply[['circulating_sol', 'nonCirculating_sol', 'total_sol']])
#print(df_accounts.head())
joblib.dump(df_supply, 'df_supply.joblib')

     circulating_sol  nonCirculating_sol          total_sol
0 519444081.87973523   81207422.19001375 600651504.06974900


['df_supply.joblib']

###  Transactions Per Second (TPS)



In [135]:
def get_tps():
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getRecentPerformanceSamples",
        "params": [1]  # Fetch 1 sample
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        sample = data["result"][0]
        tps = sample["numTransactions"] / sample["samplePeriodSecs"]
        return tps
    else:
        print("Error fetching TPS:", response.text)
        return 0

# Fetch TPS
tps = get_tps()
df_tps = pd.DataFrame({"timestamp": [pd.Timestamp.utcnow()], "tps": [tps]})
print("TPS DataFrame:")
print(df_tps.head())

joblib.dump(df_tps, 'df_tps.joblib')

TPS DataFrame:
                         timestamp           tps
0 2025-05-13 01:26:37.949912+00:00 4214.68333333


['df_tps.joblib']

### SOL Price (Using CoinGecko)




In [136]:
def get_sol_price():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=solana&vs_currencies=usd"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data["solana"]["usd"]
    else:
        print("Error fetching SOL price:", response.text)
        return 0

# Fetch SOL price
sol_price = get_sol_price()
df_price = pd.DataFrame({"timestamp": [pd.Timestamp.utcnow()], "sol_price_usd": [sol_price]})
print("SOL Price DataFrame:")
print(df_price.head())

joblib.dump(df_price, 'df_price.joblib')

SOL Price DataFrame:
                         timestamp  sol_price_usd
0 2025-05-13 01:26:38.610536+00:00   171.63000000


['df_price.joblib']

In [137]:
def get_recent_priority_fees():
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getRecentPrioritizationFees",
        "params": [[]]  # Empty accounts for global fees
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        fees = [f["prioritizationFee"] for f in data["result"]]
        avg_fee = sum(fees) / len(fees) if fees else 0  # Micro-lamports
        return avg_fee / 1e6  # Convert to lamports
    else:
        print("Error fetching priority fees:", response.text)
        return 0

# Fetch average priority fee
avg_priority_fee = get_recent_priority_fees()
base_fee = 5000  # Static base fee per signature
avg_fee = base_fee + avg_priority_fee  # Total average fee
df_fees = pd.DataFrame({
    "timestamp": [pd.Timestamp.utcnow()],
    "avg_fee_lamports": [avg_fee],
    "avg_fee_sol": [avg_fee / 1e9],
    "avg_fee_usd": [avg_fee / 1e9 * sol_price]
})
print("Network Fees DataFrame:")
print(df_fees)


joblib.dump(df_fees, 'df_fees.joblib')

Network Fees DataFrame:
                         timestamp  avg_fee_lamports  avg_fee_sol  avg_fee_usd
0 2025-05-13 01:26:39.220266+00:00     5000.00000000   0.00000500   0.00085815


['df_fees.joblib']

In [138]:


# Assuming VALIDATORS is already defined
# Example: VALIDATORS = "your-secret-api-token"

NETWORK = "mainnet"  # Change to 'testnet' or 'pythnet' as needed
BASE_URL = f"https://www.validators.app/api/v1/validators/{NETWORK}.json"

HEADERS = {
    "Token": VALIDATORS_API_KEY
}

def fetch_all_validators(with_history=False):
    params = {}
    if with_history:
        params["with_history"] = "true"
    
    response = requests.get(BASE_URL, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        validators = response.json()
        print(f"Fetched {len(validators)} validators.")
        return pd.DataFrame(validators)
    else:
        print(f"Error fetching validator data: {response.status_code} - {response.text}")
        return pd.DataFrame()

# Usage
df_validators = fetch_all_validators(with_history=True)

# Display the first few rows
print(df_validators.head())

joblib.dump(df_validators, 'df_validators.joblib')


Fetched 1265 validators.
   network                                       account                     name keybase_id                     www_url  \
0  mainnet  2gDeeRa3mwPPtw1CMWPkEhRWo9v5izNBBfEXanr8uibX                     NuFi    nufi_vl               https://nu.fi   
1  mainnet  8n4pc4sCJtBeLfJdGyJn6EcZuhtfTiepRa9ExdJFdmEN                Zeta Node                   https://zeta.markets   
2  mainnet   juigBT2qetpYpf1iwgjaiWTjryKkY3uUTVAnRFKkqY6               ProStaking                 https://ProStaking.com   
3  mainnet  6gnbmed7kzwQVQ7ghsjgEuCoYmGeWciV2qCwni6WS6HU  Capital Alliance 0% Fee                  https://capitall.app/   
4  mainnet  3WDh9HgusCujDmXCVhophLrHvoKHQd1Sd4uFHz1Awo35           Absolute Guard             https://github.com/a-guard   

                                                                                                                                                                                                                                      

['df_validators.joblib']

In [139]:
df_validators.columns.to_list()

['network',
 'account',
 'name',
 'keybase_id',
 'www_url',
 'details',
 'avatar_url',
 'created_at',
 'updated_at',
 'admin_warning',
 'jito',
 'jito_commission',
 'stake_pools_list',
 'is_active',
 'avatar_file_url',
 'active_stake',
 'authorized_withdrawer_score',
 'commission',
 'data_center_concentration_score',
 'delinquent',
 'published_information_score',
 'root_distance_score',
 'security_report_score',
 'skipped_slot_score',
 'skipped_after_score',
 'software_version',
 'software_version_score',
 'stake_concentration_score',
 'consensus_mods_score',
 'vote_latency_score',
 'total_score',
 'vote_distance_score',
 'ip',
 'data_center_key',
 'autonomous_system_number',
 'latitude',
 'longitude',
 'data_center_host',
 'vote_account',
 'epoch_credits',
 'epoch',
 'skipped_slots',
 'skipped_slot_percent',
 'ping_time',
 'url']

# Epochs

In [140]:
BASE_URL = 'https://www.validators.app/api/v1/epochs/mainnet.json'  # Replace with the desired network ('mainnet', 'testnet', 'pythnet')

# Headers with the API token for authentication
HEADERS = {
    'Token': VALIDATORS_API_KEY
}

# Make the API request
def get_epoch_data():
    params = {'per': 50, 'page': 1}  # Fetch up to 50 epochs, page 1 (adjust as needed)
    response = requests.get(BASE_URL, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data['epochs']
    else:
        print(f"Error fetching epoch data: {response.text}")
        return []

# Get epoch data
epoch_data = get_epoch_data()

# Convert the data to a pandas DataFrame
df_epochs = pd.DataFrame(epoch_data)

# Display the first few rows of the DataFrame
print(df_epochs) #.head())

joblib.dump(df_epochs, 'df_epochs.joblib')

    epoch  starting_slot  slots_in_epoch  network                created_at            total_rewards          total_active_stake
0     786      339552000          432000  mainnet  2025-05-12T15:30:43.000Z                      NaN                         NaN
1     785      339120000          432000  mainnet  2025-05-10T16:10:25.000Z 149125956701354.00000000 390905360209121664.00000000
2     784      338688000          432000  mainnet  2025-05-08T16:51:08.000Z 148872367316200.00000000 390601341255544064.00000000
3     783      338256000          432000  mainnet  2025-05-06T17:20:47.000Z 149356863823391.00000000 391590155188303488.00000000
4     782      337824002          432000  mainnet  2025-05-04T18:04:09.000Z 149294269842450.00000000 392310799006432960.00000000
5     781      337392000          432000  mainnet  2025-05-02T18:48:09.000Z 149600376018593.00000000 391208956635595392.00000000
6     780      336960000          432000  mainnet  2025-04-30T19:23:26.000Z 149328698516297.00000

['df_epochs.joblib']

# Exploratory Data Analysis

In [141]:
df_expanded.head(n=1)


,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,epoch,credits,previous_credits,credits_earned,votePubkey
5852,2025-05-13 01:26:26.561946+00:00,138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT,0,28931907684368,339642420,339642389,True,782,663485837,656598333,6887504,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz


In [142]:
df_epochs.head()

,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake
0,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN
1,785,339120000,432000,mainnet,2025-05-10T16:10:25.000Z,149125956701354.00000000,390905360209121664.00000000
2,784,338688000,432000,mainnet,2025-05-08T16:51:08.000Z,148872367316200.00000000,390601341255544064.00000000
3,783,338256000,432000,mainnet,2025-05-06T17:20:47.000Z,149356863823391.00000000,391590155188303488.00000000
4,782,337824002,432000,mainnet,2025-05-04T18:04:09.000Z,149294269842450.00000000,392310799006432960.00000000


In [143]:
print(df_epochs["epoch"].dtype)

int64


In [144]:
print(df_expanded["epoch"].dtype)

int64


In [145]:
df_expanded["activatedStake_SOL"] = df_expanded["activatedStake"] / 1e9
df_expanded["activatedStake_USD"] = df_expanded["activatedStake_SOL"] * sol_price
df_expanded.head(n=1)

,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,epoch,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD
5852,2025-05-13 01:26:26.561946+00:00,138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT,0,28931907684368,339642420,339642389,True,782,663485837,656598333,6887504,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,28931.90768437,4965583.31586808


In [146]:
df_merged = pd.merge(df_epochs, df_expanded, on="epoch", how="inner")
df_merged["vote_account"] = df_merged["votePubkey"]
df_merged.head()

,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD,vote_account
0,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT,0,28931907684368,339642420,339642389,True,685616283,684174907,1441376,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,28931.90768437,4965583.31586808,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz
1,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,13cm6z7ajighVFYN1aR2hPQ3Rhp4QJenDbHGRmps9P1n,0,226823548737864,339642419,339642388,True,663327038,661885711,1441327,F82nmpcZMdHtMVsLtAGByPavdN5WuEX1hjNwzs3UFuwq,226823.54873786,38929725.66987960,F82nmpcZMdHtMVsLtAGByPavdN5WuEX1hjNwzs3UFuwq
2,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1EWZm7aZYxfZHbyiELXtTgN1yT2vU1HF9d8DWswX2Tp,5,37652834688596,339642419,339642388,True,784327666,782886384,1441282,HG7a8fgjTkQhGFTPukbTdf5FCwxVVjKzkbo6ToNswTXH,37652.83468860,6462356.01760373,HG7a8fgjTkQhGFTPukbTdf5FCwxVVjKzkbo6ToNswTXH
3,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1KXvrkPXwkGF6NK1zyzVuJqbXfpenPVPP6hoiK9bsK3,0,278687889297847,339642419,339642388,True,658574294,657132918,1441376,1KXz4xKV2viJCGpxqnQqdf2J45vQr5USdmtcJLTaHkm,278687.88929785,47831202.44018948,1KXz4xKV2viJCGpxqnQqdf2J45vQr5USdmtcJLTaHkm
4,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1MuaDGhuN7KRqvsupUcYmq9u1YRh1pp38hu1WV2WC6S,0,94598521531858,339642419,339642388,True,601106199,599664944,1441255,4z9rbspUBsnZmTQbWSSPETkXmWHfhzQXXc289Z3m6XcJ,94598.52153186,16235944.25051279,4z9rbspUBsnZmTQbWSSPETkXmWHfhzQXXc289Z3m6XcJ


In [147]:
df_merged['total_reward_SOL'] = df_merged['total_rewards'] / 1e9
df_merged['total_active_stake_SOL'] = df_merged['total_active_stake'] / 1e9

In [148]:
df_merged

,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD,vote_account,total_reward_SOL,total_active_stake_SOL
0,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT,0,28931907684368,339642420,339642389,True,685616283,684174907,1441376,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,28931.90768437,4965583.31586808,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,NaN,NaN
1,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,13cm6z7ajighVFYN1aR2hPQ3Rhp4QJenDbHGRmps9P1n,0,226823548737864,339642419,339642388,True,663327038,661885711,1441327,F82nmpcZMdHtMVsLtAGByPavdN5WuEX1hjNwzs3UFuwq,226823.54873786,38929725.66987960,F82nmpcZMdHtMVsLtAGByPavdN5WuEX1hjNwzs3UFuwq,NaN,NaN
2,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1EWZm7aZYxfZHbyiELXtTgN1yT2vU1HF9d8DWswX2Tp,5,37652834688596,339642419,339642388,True,784327666,782886384,1441282,HG7a8fgjTkQhGFTPukbTdf5FCwxVVjKzkbo6ToNswTXH,37652.83468860,6462356.01760373,HG7a8fgjTkQhGFTPukbTdf5FCwxVVjKzkbo6ToNswTXH,NaN,NaN
3,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1KXvrkPXwkGF6NK1zyzVuJqbXfpenPVPP6hoiK9bsK3,0,278687889297847,339642419,339642388,True,658574294,657132918,1441376,1KXz4xKV2viJCGpxqnQqdf2J45vQr5USdmtcJLTaHkm,278687.88929785,47831202.44018948,1KXz4xKV2viJCGpxqnQqdf2J45vQr5USdmtcJLTaHkm,NaN,NaN
4,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1MuaDGhuN7KRqvsupUcYmq9u1YRh1pp38hu1WV2WC6S,0,94598521531858,339642419,339642388,True,601106199,599664944,1441255,4z9rbspUBsnZmTQbWSSPETkXmWHfhzQXXc289Z3m6XcJ,94598.52153186,16235944.25051279,4z9rbspUBsnZmTQbWSSPETkXmWHfhzQXXc289Z3m6XcJ,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6580,769,332208000,432000,mainnet,2025-04-08T23:44:04.000Z,150140890931048.00000000,383083844124170560.00000000,2025-05-13 01:26:26.561946+00:00,9YN6VSNh5mzNGPaHTq13Y69Uu3Dc8sQEZgQFaS7YNi7d,0,999023721,338544474,338544443,True,1996119,272456,1723663,7yBAzmPzK4oNWs6PTnrSGJzGR4kYGDC2323nhuyKUZ6B,0.99902372,171.46244124,7yBAzmPzK4oNWs6PTnrSGJzGR4kYGDC2323nhuyKUZ6B,150140.89093105,383083844.12417054
6581,766,330912000,432000,mainnet,2025-04-03T00:04:01.000Z,151035887828837.00000000,384502142003974336.00000000,2025-05-13 01:26:26.561946+00:00,Syrd2b19zpvDrSTSkUzRUoPm9xpJkPtfKj77oGDCUHN,0,32206236567,339283336,339283305,True,461129027,460324854,804173,Syrd4L1eGcZdhRGoB9wb4aJKKJKv9gMudZLMnXdV7AR,32.20623657,5527.55638199,Syrd4L1eGcZdhRGoB9wb4aJKKJKv9gMudZLMnXdV7AR,151035.88782884,384502142.00397432
6582,746,322272000,432000,mainnet,2025-02-22T04:45:22.000Z,152809126746356.00000000,383879948504408320.00000000,2025-05-13 01:26:26.561946+00:00,9SJdwWQ1YgRdYcQ9NinWUussJNX4MUKdMEy2SpMAfe67,0,1000000,337647080,337647049,True,4472689,4199203,273486,5fxz3nazxtC3pW119KPV6qEELuBSSGxvo152b4UyW9Lj,0.00100000,0.17163000,5fxz3nazxtC3pW119KPV6qEELuBSSGxvo152b4UyW9Lj,152809.12674636,383879948.50440830
6583,745,321840000,432000,mainnet,2025-02-20T05:14:55.000Z,153015323340118.00000000,384476729594204992.00000000,2025-05-13 01:26:26.561946+00:00,9SJdwWQ1YgRdYcQ9NinWUussJNX4MUKdMEy2SpMAfe67,0,1000000,337647080,337647049,True,4199203,126181,4073022,5fxz3nazxtC3pW119KPV6qEELuBSSGxvo152b4UyW9Lj,0.00100000,0.17163000,5fxz3nazxtC3pW119KPV6qEELuBSSGxvo152b4UyW9Lj,153015.32334012,384476729.59420496


In [149]:
df_merged.columns.tolist()

['epoch',
 'starting_slot',
 'slots_in_epoch',
 'network',
 'created_at',
 'total_rewards',
 'total_active_stake',
 'timestamp',
 'nodePubkey',
 'commission',
 'activatedStake',
 'lastVote',
 'rootSlot',
 'epochVoteAccount',
 'credits',
 'previous_credits',
 'credits_earned',
 'votePubkey',
 'activatedStake_SOL',
 'activatedStake_USD',
 'vote_account',
 'total_reward_SOL',
 'total_active_stake_SOL']

In [150]:
# BASE_URL = f"https://www.validators.app/api/v1/validators/{NETWORK}.json"
# def fetch_all_validators(with_history=False):
#     params = {"with_history": "true"} if with_history else {}
#     response = requests.get(BASE_URL, headers={"Token": VALIDATORS_API_KEY}, params=params)
#     return pd.DataFrame(response.json()) if response.status_code == 200 else pd.DataFrame()
# df_validators = fetch_all_validators(with_history=True)



In [151]:
df_validators.head(n=1)

,network,account,name,keybase_id,www_url,details,avatar_url,created_at,updated_at,admin_warning,jito,jito_commission,stake_pools_list,is_active,avatar_file_url,active_stake,authorized_withdrawer_score,commission,data_center_concentration_score,delinquent,published_information_score,root_distance_score,security_report_score,skipped_slot_score,skipped_after_score,software_version,software_version_score,stake_concentration_score,consensus_mods_score,vote_latency_score,total_score,vote_distance_score,ip,data_center_key,autonomous_system_number,latitude,longitude,data_center_host,vote_account,epoch_credits,epoch,skipped_slots,skipped_slot_percent,ping_time,url
0,mainnet,2gDeeRa3mwPPtw1CMWPkEhRWo9v5izNBBfEXanr8uibX,NuFi,nufi_vl,https://nu.fi,Try NuFi wallet!,https://s3.amazonaws.com/keybase_processed_uploads/a547656e4d522f6c0eec7508ff61a605_360_360.jpg,2021-07-22 12:30:46 UTC,2025-04-24 03:40:06 UTC,None,True,800.00000000,"[Jito, Jpool, Aero]",True,https://prod-validators.nyc3.digitaloceanspaces.com/d297r2amrdfhvfxau8vxj17b9uyu,334760803209436,0,0,0,False,2,2,1,2,2,2.1.21,2,0,0,2.00000000,13,2,109.74.144.98,29405-SK-Šaľa,29405,48.1592,17.8834,None,3Xn3K5zeTzs4sURj1PCRaF6rvFg2hPgT49B4SiM998f7,1441686.00000000,786.00000000,0.00000000,0.0,NaN,https://www.validators.app/api/v1/validators/mainnet/2gDeeRa3mwPPtw1CMWPkEhRWo9v5izNBBfEXanr8uibX


In [152]:
# 1. Separate static validator metadata
static_validator_cols = [
    'vote_account', 'name', 'keybase_id', 'www_url',
    'details', 'avatar_url'
]
df_validators_static = df_validators[static_validator_cols].drop_duplicates()

# 2. Merge with historical data using ONLY vote_account
df_final = pd.merge(
    df_merged,
    df_validators_static,
    on='vote_account',
    how='left'
)


df_final

,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake,timestamp,nodePubkey,commission,activatedStake,lastVote,rootSlot,epochVoteAccount,credits,previous_credits,credits_earned,votePubkey,activatedStake_SOL,activatedStake_USD,vote_account,total_reward_SOL,total_active_stake_SOL,name,keybase_id,www_url,details,avatar_url
0,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,138KHwTqKNWGLoo8fK5i8UxYtwoC5tC8o7M9rY1CDEjT,0,28931907684368,339642420,339642389,True,685616283,684174907,1441376,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,28931.90768437,4965583.31586808,ASfKFAKz6fH4eip1jdLGt5Ym954kU9KYnwq2Csn9ogSz,NaN,NaN,Nodes24.io 💎Jito 0% MEV💥,,https://nodes24.io/,Nodes24 is a reliable staking validator for your crypto,https://nodes24.io/images/apple-touch-icon.png
1,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,13cm6z7ajighVFYN1aR2hPQ3Rhp4QJenDbHGRmps9P1n,0,226823548737864,339642419,339642388,True,663327038,661885711,1441327,F82nmpcZMdHtMVsLtAGByPavdN5WuEX1hjNwzs3UFuwq,226823.54873786,38929725.66987960,F82nmpcZMdHtMVsLtAGByPavdN5WuEX1hjNwzs3UFuwq,NaN,NaN,LAB517,vladmsq,,,https://s3.amazonaws.com/keybase_processed_uploads/a7648404e2054a7f4568e0ef6fcaad05_360_360.jpg
2,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1EWZm7aZYxfZHbyiELXtTgN1yT2vU1HF9d8DWswX2Tp,5,37652834688596,339642419,339642388,True,784327666,782886384,1441282,HG7a8fgjTkQhGFTPukbTdf5FCwxVVjKzkbo6ToNswTXH,37652.83468860,6462356.01760373,HG7a8fgjTkQhGFTPukbTdf5FCwxVVjKzkbo6ToNswTXH,NaN,NaN,clockchain,clockchain,https://clockchainstake.tk,Clockchain technologies,https://s3.amazonaws.com/keybase_processed_uploads/e74fe161942373aa3f96094b4fa70505_360_360.jpg
3,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1KXvrkPXwkGF6NK1zyzVuJqbXfpenPVPP6hoiK9bsK3,0,278687889297847,339642419,339642388,True,658574294,657132918,1441376,1KXz4xKV2viJCGpxqnQqdf2J45vQr5USdmtcJLTaHkm,278687.88929785,47831202.44018948,1KXz4xKV2viJCGpxqnQqdf2J45vQr5USdmtcJLTaHkm,NaN,NaN,1000X.sh,1000xstake,https://1000x.sh,Stake with the best,https://s3.amazonaws.com/keybase_processed_uploads/a292977de8e5dd1e12d03eb9c26a7e05_360_360.jpg
4,786,339552000,432000,mainnet,2025-05-12T15:30:43.000Z,NaN,NaN,2025-05-13 01:26:26.561946+00:00,1MuaDGhuN7KRqvsupUcYmq9u1YRh1pp38hu1WV2WC6S,0,94598521531858,339642419,339642388,True,601106199,599664944,1441255,4z9rbspUBsnZmTQbWSSPETkXmWHfhzQXXc289Z3m6XcJ,94598.52153186,16235944.25051279,4z9rbspUBsnZmTQbWSSPETkXmWHfhzQXXc289Z3m6XcJ,NaN,NaN,Mercurial Validator,,https://mercurialvalidator.com,,https://arweave.net/KOfFDGNUYYH2uBe_e-5gi3xXHIBp7JwHac02RZBi-TI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,769,332208000,432000,mainnet,2025-04-08T23:44:04.000Z,150140890931048.00000000,383083844124170560.00000000,2025-05-13 01:26:26.561946+00:00,9YN6VSNh5mzNGPaHTq13Y69Uu3Dc8sQEZgQFaS7YNi7d,0,999023721,338544474,338544443,True,1996119,272456,1723663,7yBAzmPzK4oNWs6PTnrSGJzGR4kYGDC2323nhuyKUZ6B,0.99902372,171.46244124,7yBAzmPzK4oNWs6PTnrSGJzGR4kYGDC2323nhuyKUZ6B,150140.89093105,383083844.12417054,NaN,NaN,NaN,NaN,NaN
6596,766,330912000,432000,mainnet,2025-04-03T00:04:01.000Z,151035887828837.00000000,384502142003974336.00000000,2025-05-13 01:26:26.561946+00:00,Syrd2b19zpvDrSTSkUzRUoPm9xpJkPtfKj77oGDCUHN,0,32206236567,339283336,339283305,True,461129027,460324854,804173,Syrd4L1eGcZdhRGoB9wb4aJKKJKv9gMudZLMnXdV7AR,32.20623657,5527.55638199,Syrd4L1eGcZdhRGoB9wb4aJKKJKv9gMudZLMnXdV7AR,151035.88782884,384502142.00397432,NaN,NaN,NaN,NaN,NaN
6597,746,322272000,432000,mainnet,2025-02-22T04:45:22.000Z,152809126746356.00000000,383879948504408320.00000000,2025-05-13 01:26:26.561946+00:00,9SJdwWQ1YgRdYcQ9NinWUussJNX4MUKdMEy2SpMAfe67,0,1000000,337647080,337647049,True,4472689,4199203,273486,5fxz3nazxtC3pW119KPV6qEEL

In [153]:
df_final_sorted = df_final.sort_values(by='epoch', ascending=False).reset_index(drop=True)

df_final_sorted

%store df_final_sorted

Stored 'df_final_sorted' (DataFrame)


/Users/olaoluwatunmise/miniconda3/envs/solana-predict/lib/python3.10/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/df_final_sorted requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [154]:
import joblib

# Preferred: use .joblib extension
joblib.dump(df_final_sorted, 'df_final_sorted.joblib')

['df_final_sorted.joblib']

In [155]:
# df_final['name'].isna().sum()

In [156]:
df_final_sorted.columns.tolist()

['epoch',
 'starting_slot',
 'slots_in_epoch',
 'network',
 'created_at',
 'total_rewards',
 'total_active_stake',
 'timestamp',
 'nodePubkey',
 'commission',
 'activatedStake',
 'lastVote',
 'rootSlot',
 'epochVoteAccount',
 'credits',
 'previous_credits',
 'credits_earned',
 'votePubkey',
 'activatedStake_SOL',
 'activatedStake_USD',
 'vote_account',
 'total_reward_SOL',
 'total_active_stake_SOL',
 'name',
 'keybase_id',
 'www_url',
 'details',
 'avatar_url']

In [157]:
latest_epoch = df_final_sorted['epoch'].max()

# Filter out the rows with the latest epoch where total_rewards and total_active_stake are NaN
df_cleaned = df_final_sorted[~((df_final_sorted['epoch'] == latest_epoch) & df_final_sorted['total_rewards'].isna() & df_final_sorted['total_active_stake'].isna())].reset_index(drop=True)

df_cleaned.head(n=5)

joblib.dump(df_cleaned, 'df_cleaned.joblib')


['df_cleaned.joblib']